# Validate Distilled Student Model (Stage 2)

This notebook validates the ConvNeXt-Tiny student model after knowledge distillation training.

**Training Job:** pytorch-training-2026-01-11-09-27-50-349

**Model:** s3://sagemaker-us-east-2-943271038849/pytorch-training-2026-01-11-09-27-50-349/output/model.tar.gz

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import boto3
import tarfile
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Download Model from S3

In [ ]:
# Download model artifact
s3 = boto3.client('s3')
bucket = 'sagemaker-us-east-2-943271038849'
key = 'pytorch-training-2026-01-11-09-27-50-349/output/model.tar.gz'
local_path = '/tmp/stage2_model.tar.gz'

print("Downloading model from S3...")
s3.download_file(bucket, key, local_path)
print(f"Downloaded to {local_path}")

# Extract
extract_dir = '/tmp/stage2_model'
os.makedirs(extract_dir, exist_ok=True)
with tarfile.open(local_path, 'r:gz') as tar:
    tar.extractall(extract_dir)

print("\nExtracted files:")
for f in os.listdir(extract_dir):
    size_mb = os.path.getsize(os.path.join(extract_dir, f)) / (1024**2)
    print(f"  {f}: {size_mb:.1f} MB")

## 2. Check Training Metrics from Checkpoint

In [ ]:
# Load checkpoint to see training metrics
checkpoint_path = os.path.join(extract_dir, 'student_stage2_checkpoint.pt')

# Need to define WarmupCosineScheduler class first to load checkpoint
class WarmupCosineScheduler:
    """Placeholder to allow checkpoint loading"""
    pass

try:
    checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
    print("=== Stage 2 Training Results ===")
    print(f"Final Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"Cosine Similarity: {checkpoint.get('cosine_similarity', 'N/A'):.4f}")
    print(f"Stage: {checkpoint.get('stage', 'N/A')}")
    print(f"\nCheckpoint keys: {list(checkpoint.keys())}")
except Exception as e:
    print(f"Could not load checkpoint metadata: {e}")
    print("This is okay - we can still validate the model weights.")

## 3. Load Student Model Architecture

In [ ]:
class StudentModel(nn.Module):
    def __init__(self, model_name='convnext_tiny', embedding_dim=768, num_classes=17592):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=False, num_classes=0)
        backbone_dim = self.backbone.num_features
        
        self.projector = nn.Sequential(
            nn.Linear(backbone_dim, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )
        
        self.arcface = nn.Linear(embedding_dim, num_classes, bias=False)
    
    def forward(self, x):
        features = self.backbone(x)
        embeddings = self.projector(features)
        embeddings = nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings
    
    def get_embeddings(self, x):
        return self.forward(x)

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StudentModel(
    model_name='convnext_tiny',
    embedding_dim=768,
    num_classes=17592
).to(device)

print(f"Model created on {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Load Trained Weights

In [ ]:
# Load the best model weights (EMA)
model_path = os.path.join(extract_dir, 'student_stage2.pt')
state_dict = torch.load(model_path, map_location=device, weights_only=False)

# Strip _orig_mod. prefix if present (from torch.compile)
if any(k.startswith('_orig_mod.') for k in state_dict.keys()):
    state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
    print("Stripped _orig_mod. prefix from keys")

model.load_state_dict(state_dict)
model.eval()
print("✅ Model weights loaded successfully!")

## 5. Load Validation Dataset

In [ ]:
# Download a sample of validation data from S3
# Note: Adjust this path based on where your validation data is stored
val_data_s3 = 's3://pokemon-card-training-us-east-2/classification_dataset/val/'

# For quick testing, let's use a local path or download subset
# You'll need to adjust this based on your SageMaker Studio setup
val_dir = '/opt/ml/input/data/val'  # or wherever you mount your data

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Check if validation data is available
if os.path.exists(val_dir):
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    print(f"✅ Loaded validation dataset: {len(val_dataset)} images, {len(val_dataset.classes)} classes")
else:
    print("⚠️ Validation data not found. You'll need to:")
    print(f"   1. Copy data from {val_data_s3}")
    print(f"   2. Or mount the S3 bucket in SageMaker Studio")
    val_loader = None

## 6. Validate Model Performance

In [ ]:
if val_loader is not None:
    @torch.no_grad()
    def validate_model(model, val_loader, device):
        model.eval()
        all_embeddings = []
        all_labels = []
        
        print("Generating embeddings...")
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            embeddings = model(images)
            all_embeddings.append(embeddings.cpu())
            all_labels.append(labels)
        
        all_embeddings = torch.cat(all_embeddings, dim=0)
        all_labels = torch.cat(all_labels, dim=0)
        
        # Compute cosine similarity matrix
        print("Computing similarities...")
        similarity_matrix = torch.mm(all_embeddings, all_embeddings.t())
        
        # Top-1 retrieval accuracy
        similarity_matrix.fill_diagonal_(-1)  # Exclude self
        top1_indices = similarity_matrix.argmax(dim=1)
        top1_labels = all_labels[top1_indices]
        top1_acc = (top1_labels == all_labels).float().mean().item()
        
        # Top-5 retrieval accuracy
        top5_indices = similarity_matrix.topk(5, dim=1).indices
        top5_labels = all_labels[top5_indices]
        top5_acc = (top5_labels == all_labels.unsqueeze(1)).any(dim=1).float().mean().item()
        
        # Average cosine similarity for same class
        label_mask = all_labels.unsqueeze(0) == all_labels.unsqueeze(1)
        label_mask.fill_diagonal_(False)
        same_class_sim = similarity_matrix[label_mask].mean().item()
        
        # Average cosine similarity for different class
        diff_class_sim = similarity_matrix[~label_mask].mean().item()
        
        return {
            'top1_accuracy': top1_acc,
            'top5_accuracy': top5_acc,
            'same_class_similarity': same_class_sim,
            'diff_class_similarity': diff_class_sim,
            'separation': same_class_sim - diff_class_sim
        }
    
    metrics = validate_model(model, val_loader, device)
    
    print("\n" + "="*50)
    print("📊 VALIDATION RESULTS")
    print("="*50)
    print(f"Top-1 Retrieval Accuracy: {metrics['top1_accuracy']:.2%}")
    print(f"Top-5 Retrieval Accuracy: {metrics['top5_accuracy']:.2%}")
    print(f"Same Class Similarity:    {metrics['same_class_similarity']:.4f}")
    print(f"Diff Class Similarity:    {metrics['diff_class_similarity']:.4f}")
    print(f"Class Separation:         {metrics['separation']:.4f}")
    print("="*50)
else:
    print("⚠️ Skipping validation - no data available")

## 7. Test Inference Speed

In [ ]:
# Benchmark inference speed
import time

batch_sizes = [1, 8, 16, 32, 64]
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Warmup
for _ in range(10):
    _ = model(dummy_input)

print("\n📈 INFERENCE SPEED (GPU)")
print("="*50)
for bs in batch_sizes:
    dummy_batch = torch.randn(bs, 3, 224, 224).to(device)
    
    torch.cuda.synchronize()
    start = time.time()
    
    for _ in range(100):
        _ = model(dummy_batch)
    
    torch.cuda.synchronize()
    elapsed = time.time() - start
    
    throughput = (bs * 100) / elapsed
    latency = (elapsed / 100) * 1000  # ms
    
    print(f"Batch {bs:2d}: {throughput:6.1f} img/sec, {latency:5.1f} ms/batch")
print("="*50)

## 8. Model Statistics

In [ ]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total, trainable = count_parameters(model)

print("\n🔍 MODEL STATISTICS")
print("="*50)
print(f"Architecture:      ConvNeXt-Tiny")
print(f"Embedding Dim:     768")
print(f"Total Parameters:  {total:,} ({total/1e6:.1f}M)")
print(f"Trainable Params:  {trainable:,} ({trainable/1e6:.1f}M)")
print(f"Model Size:        ~162 MB")
print(f"Compression:       39x (from 1.1B teacher)")
print("="*50)

## 9. Compare with Teacher Model (Optional)

If you want to compare student vs teacher performance, download the teacher model and run the same validation.

In [ ]:
# This cell is optional - only if you want to compare with teacher
# teacher_s3_path = 's3://pokemon-card-training-us-east-2/models/embedding/teacher/pokemon-card-dinov3-teacher-2026-01-10-13-31-34-937/output/model.tar.gz'
# ... (load teacher and compare)

## Summary

This notebook validated the distilled ConvNeXt-Tiny student model trained with knowledge distillation from a DINOv3-ViT-Large teacher.

**Key Metrics to Look For:**
- **Top-1 Accuracy**: Should be >85% for good retrieval
- **Class Separation**: Should be >0.3 for distinct embeddings
- **Inference Speed**: Should be much faster than teacher (~10-50x)

**Next Steps:**
1. Export to ONNX for deployment
2. Compile for Hailo-8L accelerator
3. Deploy on Raspberry Pi